In [1]:
import json, requests
import Levenshtein as lev
import unicodedata
import pandas as pd

In [2]:

ID = "aaf20ac1"
key = "7bd88c15c64569da8de3c414ace8d236"

baseURL = "http://api.yummly.com/v1/api/recipes?_app_id="+ID+"&_app_key="+key+"&q="

In [3]:
def convertName(name):
    converted = unicodedata.normalize('NFKD', name).encode('ascii','ignore')
    return converted.lower()

def bestMatch(query, response):
    tmpQuery = query.lower()
    levDistances = [lev.distance(convertName(match["recipeName"]), tmpQuery) for match in response["matches"]]
    return response["matches"][levDistances.index(min(levDistances))]
    

In [15]:
def searchFromList():
    responses = []
    foodList = pd.read_csv("dummyFood.csv", sep=',')
    for query in foodList['FoodName']:
        params = query.lower().replace(" ", "+")
        url = baseURL+params+""
        response = json.loads(requests.get(url).text)
        candidate = bestMatch(query, response)
        candidate['ingredients']
        responses.append(response)
    return responses
    
responses = searchFromList()

In [18]:
for response in responses:
    print response['matches'][0]['ingredients']
    print 

[u'flour', u'pizza doughs', u'canned tomatoes', u'low-fat mozzarella cheese', u'basil leaves', u'salt', u'olive oil']

[u'canola oil', u'finely chopped onion', u'chopped celery', u'diced tomatoes', u'water', u'brown sugar', u'salt', u'dried basil', u'dried oregano', u'ground pepper']

[u'yellow onion', u'beef broth', u'baguette', u'bay leaf', u'gruyere cheese', u'unsalted butter', u'all-purpose flour', u'extra-virgin olive oil', u'balsamic vinegar', u'worcestershire sauce', u'kosher salt', u'freshly ground pepper', u'chives']

[u'sun-dried tomatoes', u'chicken breasts', u'garlic cloves', u'salt', u'paprika', u'half & half', u'asiago', u'penne pasta', u'fresh spinach']

[u'lean ground beef', u'whole wheat spaghetti', u'marinara sauce', u'chopped onion', u'carrots', u'chopped celery', u'minced garlic', u'oregano', u'extra-virgin olive oil', u'sea salt', u'black pepper']

